In [ ]:
# general libraries
import time
import math
import numpy as np
from shapely import geometry
from shapely.geometry import Polygon
import matplotlib
import matplotlib.pyplot as plt
import pickle
import os
import geopandas as gpd
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import pandas as pd
import dill
import geopandas
import geopandas as gpd
import matplotlib.colors as colors


# JAX
import jax
import jax.numpy as jnp
from jax import random, lax, jit, ops
from jax.experimental import stax


# Numpyro
import numpyro
from numpyro.infer import SVI, MCMC, NUTS, init_to_median, Predictive, RenyiELBO
import numpyro.distributions as dist
from numpyro.diagnostics import hpdi
from numpyro import optim

print(jax.__version__)
print(numpyro.__version__)

## Read shape files

In [ ]:
shape_new = gpd.read_file('../data/ken_adm_iebc_20191031_shp/ken_admbnda_adm1_iebc_20191031.shp')
shape_new = shape_new[['ADM1_EN', 'geometry']]
shape_new['new'] = 1
shape_new.head(2)

In [ ]:
shape_old = gpd.read_file('../data/ke_district_boundaries/ke_district_boundaries.shp')
shape_old = shape_old.rename({'DISTNAME': 'ADM1_EN'}, axis='columns')
shape_old['new'] = 0
shape_old.head(2)

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, sharex=True, sharey=True)
shape_new.plot(ax=ax1, column='ADM1_EN')
shape_old.plot(ax=ax2, column='ADM1_EN')

In [ ]:
shape_old.boundary.plot(linewidth=0.5,  edgecolor='black', figsize=(6,6))
#plt.rcParams['figure.dpi'] = 300
#plt.rcParams['savefig.dpi'] = 300
#plt.savefig('res/boundaries_before2010.pdf')

In [ ]:
shape_new.boundary.plot(linewidth=0.5,  edgecolor='black', figsize=(6,6))
#plt.rcParams['figure.dpi'] = 300
#plt.rcParams['savefig.dpi'] = 300
#plt.savefig('res/boundaries_after2010.pdf')

In [ ]:
# capitalise unit names
names_shape_new = shape_new['ADM1_EN'].values
shape_new['ADM1_EN'] =  np.array([names_shape_new[i].upper() for i in range(len(names_shape_new))])
shape_new.head(5)

In [ ]:
names_shape_new = shape_new['ADM1_EN'].values
names_shape_new

In [ ]:
# capitalise unit names
names_shape_old = shape_old['ADM1_EN'].values
shape_old['ADM1_EN'] =  np.array([names_shape_old[i].upper() for i in range(len(names_shape_old))])
shape_old.head(5)

In [ ]:
names_shape_old = shape_old['ADM1_EN'].values
names_shape_old

# Malaria data

In [ ]:
infections_new = pd.read_csv("../data/dhs/KE_prev_data_admin1_after2010.csv")
infections_new = infections_new[['unit', 'usage', 'num_rdt', 'num_rdt_positive', 'prev_estimate']]
infections_new = infections_new.rename({'unit': 'area'}, axis='columns')
infections_new = infections_new.sort_values(by=['area'])
infections_new.head(5)

In [ ]:
infections_old = pd.read_csv("../data/dhs/KE_prev_data_admin1_before2010.csv")
infections_old.head(2)

In [ ]:
infections_old = infections_old[['unit', 'usage', 'num_rdt', 'num_rdt_positive', 'prev_estimate']]
infections_old = infections_old.rename({'unit': 'area'}, axis='columns')
infections_old = infections_old.sort_values(by=['area'])
infections_old.head(2)

## Merge infections and shape

### After 2010

In [ ]:
names_infections_new = infections_new.area.values
shape_new = shape_new.rename({'ADM1_EN': 'area'}, axis='columns')
shape_new = shape_new.sort_values(by=['area'])
shape_new.head(3)
names_shape_new = shape_new['area'].values
print(names_infections_new)
print(names_shape_new)

In [ ]:
names_shape_new_lst = list(names_shape_new)
lst = [names_shape_new_lst[i] not in names_infections_new for i in range(len(names_shape_new))]
[i for i in range(len(lst)) if lst[i] == True]

In [ ]:
names_infections_new[[4, 28, 40, 41]]

In [ ]:
names_shape_new[4] = 'ELGEYO MARAKWET'
names_shape_new[28] = 'MURANGA'
names_shape_new[40] = 'THARAKA'
names_shape_new[41] = 'TRANS-NZOIA'
shape_new['area'] =  names_shape_new

In [ ]:
shape_new = shape_new.merge(infections_new, how='left', left_on='area', right_on='area')
shape_new.head(3)

In [ ]:
np.sum(np.isnan(shape_new.num_rdt))

In [ ]:
s_new = shape_new.rename({'num_rdt': 'n_obs', 'num_rdt_positive': 'y', 'prev_estimate': 'estimate'}, axis='columns')
s_new['y'] = round(s_new['y']).astype(int)
s_new['n_obs'] = round(s_new['n_obs']).astype(int)
s_new.head(3)

In [ ]:
fig, ax = plt.subplots(1, 4, figsize=(12,6))
s_new.plot(column="y", ax=ax[0], legend=True)
s_new.plot(column="n_obs", ax=ax[1], legend=True)
s_new.plot(column="estimate", ax=ax[2], legend=True)
s_new.plot(column="usage", ax=ax[3], legend=True)
ax[0].set_title( 'y')
ax[1].set_title( 'n_obs')
ax[2].set_title( 'estimate')
ax[3].set_title( 'usage')

In [ ]:
del shape_new
del infections_new

### Before 2010

In [ ]:
names_infections_old = infections_old.area.values
names_infections_old

In [ ]:
shape_old = shape_old.rename({'ADM1_EN': 'area'}, axis='columns')
shape_old = shape_old.sort_values(by=['area'])
shape_old.head(3)

In [ ]:
names_shape_old = shape_old['area'].values
names_shape_old

In [ ]:
names_shape_old_lst = list(names_shape_old)
lst = [names_shape_old_lst[i] not in names_infections_old for i in range(len(names_shape_old))]
[i for i in range(len(lst)) if lst[i] == True]

In [ ]:
shape_old = shape_old.merge(infections_old, how='left', left_on='area', right_on='area')
shape_old.head(3)

In [ ]:
print(np.sum(np.isnan(shape_old.num_rdt)))
print(np.sum(np.isnan(shape_old.num_rdt_positive)))

In [ ]:
s_old = shape_old.rename({'num_rdt': 'n_obs', 'num_rdt_positive': 'y', 'prev_estimate': 'estimate'}, axis='columns')
s_old['y'] = round(s_old['y']).astype(int)
s_old['n_obs'] = round(s_old['n_obs']).astype(int)
s_old.head(3)

In [ ]:
fig, ax = plt.subplots(1, 4, figsize=(12,6))
s_old.plot(column="y", ax=ax[0], legend=True)
s_old.plot(column="n_obs", ax=ax[1], legend=True)
s_old.plot(column="estimate", ax=ax[2], legend=True)
s_old.plot(column="usage", ax=ax[3], legend=True)
ax[0].set_title( 'y')
ax[1].set_title( 'n_obs')
ax[2].set_title( 'estimate')
ax[3].set_title( 'usage')

In [ ]:
del shape_old
del infections_old

## Computational grid

In [ ]:
x_min, x_max, y_min, y_max = 33.4, 42.1, -4.8, 5.6
dy = y_max - y_min
dx = x_max - x_min
factor = dy / dx
n_x = 40
n_y = math.ceil(n_x * factor)
print(n_x)
print(n_y)
print(n_x * n_y)

x_grid = np.linspace(x_min, x_max, n_x, endpoint=True)
y_grid = np.linspace(y_min, y_max, n_y, endpoint=True)

# full coordinate arrays
x_coords, y_coords = np.meshgrid(x_grid, y_grid)
x_coords = x_coords.reshape(-1)
y_coords = y_coords.reshape(-1)

df = pd.DataFrame({'Latitude': y_coords, 'Longitude': x_coords})

grid_pts = geopandas.GeoDataFrame(df, geometry=geopandas.points_from_xy(df.Longitude, df.Latitude))

x = jnp.array([x_coords, y_coords]).transpose((1, 0))
print(x.shape)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6,6))
divider = make_axes_locatable(ax)
s_new.plot(ax=ax, color='white', edgecolor='black')
grid_pts.plot(ax=ax, marker='o', color='red', markersize=2)

#plt.rcParams['figure.dpi'] = 300
#plt.rcParams['savefig.dpi'] = 300
#plt.savefig('res/comp_grid.pdf')

In [ ]:
def pol_pts(shape, grid_pts):
    grid_pts.set_crs(epsg=4326, inplace=True);
    grid_pts.crs == shape.crs

    n_pol = len(shape.geometry)
    n_pts = len(grid_pts.geometry)

    pl_pt = np.zeros((n_pol, n_pts), dtype=int)
    
    pt_which_pol = np.zeros(n_pts, dtype=int)

    for i_pol in range(n_pol):
        pol = shape.geometry[i_pol]
        for j_pts in range(n_pts):
            pt = grid_pts.geometry[j_pts]
            if pol.contains(pt):
                pl_pt[i_pol, j_pts] = 1
                pt_which_pol[j_pts] = i_pol + 1 # note the enumeration!
                
    return([pl_pt, pt_which_pol])

In [ ]:
pol_pt_new, pt_which_pol_new = pol_pts(s_new, grid_pts)
pol_sums_new = np.sum(pol_pt_new, axis=1) # how many points in each polygon
print(all(item > 0 for item in pol_sums_new)) # in each polygon there is at least one point
grid_pts['pt_which_pol_new'] = pt_which_pol_new
grid_pts.head(2)

In [ ]:
pol_pt_old, pt_which_pol_old = pol_pts(s_old, grid_pts)
pol_sums_old = np.sum(pol_pt_old, axis=1) # how many points in each polygon
print(all(item > 0 for item in pol_sums_old)) # in each polygon there is at least one point
grid_pts['pt_which_pol_old'] = pt_which_pol_old
grid_pts.head(2)

In [ ]:
grid_pts['in_out_new'] = grid_pts['pt_which_pol_new'].apply(lambda x: 'out' if x==0 else 'in')
grid_pts['marker_size_new'] = grid_pts['pt_which_pol_new'].apply(lambda x: 1 if x==0 else 12)
n_pol_new = len(s_new.geometry)

grid_pts['in_out_old'] = grid_pts['pt_which_pol_old'].apply(lambda x: 'out' if x==0 else 'in')
grid_pts['marker_size_old'] = grid_pts['pt_which_pol_old'].apply(lambda x: 1 if x==0 else 12)
n_pol_old = len(s_old.geometry)

In [ ]:
cmap = plt.cm.prism  # define the colormap
# extract all colors from the .jet map
cmaplist = [cmap(i) for i in range(cmap.N)]
# force the first color entry to be grey
cmaplist[0] = (.5, .5, .5, 1.0)
cmap = matplotlib.colors.LinearSegmentedColormap.from_list('Custom cmap', cmaplist, cmap.N)

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, sharex=True, sharey=True)
divider = make_axes_locatable(ax)

# define the bins and normalize
bounds_old = np.linspace(min(pt_which_pol_old), max(pt_which_pol_old), n_pol_old)
norm_old = matplotlib.colors.BoundaryNorm(bounds_old, cmap.N)
s_old.plot(ax=ax1, color='white', edgecolor='black')
grid_pts.plot(ax=ax1, marker='o', column='pt_which_pol_old', markersize='marker_size_old', cmap=cmap)
ax1.title.set_text('Before 2010')

# define the bins and normalize
bounds_new = np.linspace(min(pt_which_pol_new), max(pt_which_pol_new), n_pol_new)
norm_new = matplotlib.colors.BoundaryNorm(bounds_new, cmap.N)
s_new.plot(ax=ax2, color='white', edgecolor='black')
grid_pts.plot(ax=ax2, marker='o', column='pt_which_pol_new', markersize='marker_size_new', cmap=cmap)
ax2.title.set_text('After 2010')

#plt.show();
#plt.rcParams['figure.dpi'] = 300
#plt.rcParams['savefig.dpi'] = 300
#plt.savefig('res/grid_polygons_before_after.pdf')

In [ ]:
fig, ax1 = plt.subplots(1, 1, sharex=True, sharey=True, figsize=(6,6))
divider = make_axes_locatable(ax)

# define the bins and normalize
bounds_old = np.linspace(min(pt_which_pol_old), max(pt_which_pol_old), n_pol_old)
norm_old = matplotlib.colors.BoundaryNorm(bounds_old, cmap.N)
s_old.plot(ax=ax1, color='white', edgecolor='black')
grid_pts.plot(ax=ax1, marker='o', column='pt_which_pol_old', markersize='marker_size_old', cmap=cmap)
#ax1.title.set_text('Before 2010')

# define the bins and normalize
#bounds_new = np.linspace(min(pt_which_pol_new), max(pt_which_pol_new), n_pol_new)
#norm_new = matplotlib.colors.BoundaryNorm(bounds_new, cmap.N)
#s_new.plot(ax=ax2, color='white', edgecolor='black')
#grid_pts.plot(ax=ax2, marker='o', column='pt_which_pol_new', markersize='marker_size_new', cmap=cmap)
#ax2.title.set_text('After 2010')

#plt.rcParams['figure.dpi'] = 300
#plt.rcParams['savefig.dpi'] = 300
#plt.savefig('res/grid_polygons_before.pdf')

In [ ]:
fig, ax2 = plt.subplots(1, 1, sharex=True, sharey=True, figsize=(6,6))
divider = make_axes_locatable(ax)

# define the bins and normalize
bounds_new = np.linspace(min(pt_which_pol_new), max(pt_which_pol_new), n_pol_new)
norm_new = matplotlib.colors.BoundaryNorm(bounds_new, cmap.N)
s_new.plot(ax=ax2, color='white', edgecolor='black')
grid_pts.plot(ax=ax2, marker='o', column='pt_which_pol_new', markersize='marker_size_new', cmap=cmap)
#ax2.title.set_text('After 2010')

#plt.rcParams['figure.dpi'] = 300
#plt.rcParams['savefig.dpi'] = 300
#plt.savefig('res/grid_polygons_after.pdf')

In [ ]:
s_old.head(3)

In [ ]:
s_new.head(3)

In [ ]:
type(s_new)

In [ ]:
s = pd.concat([s_old, s_new])
s.head()
print(s.shape)

# Traditional prevalence model

In [ ]:
# draw GP samples on the grid

def dist_euclid(x, z):
    x = jnp.array(x) 
    z = jnp.array(z)
    if len(x.shape)==1:
        x = x.reshape(x.shape[0], 1)
    if len(z.shape)==1:
        z = x.reshape(x.shape[0], 1)
    n_x, m = x.shape
    n_z, m_z = z.shape
    assert m == m_z
    delta = jnp.zeros((n_x,n_z))
    for d in jnp.arange(m):
        x_d = x[:,d]
        z_d = z[:,d]
        delta += (x_d[:,jnp.newaxis] - z_d)**2
    return jnp.sqrt(delta)


def exp_sq_kernel(x, z, var, length, noise, jitter=1.0e-4):
    dist = dist_euclid(x, z)
    deltaXsq = jnp.power(dist/ length, 2.0)
    k = var * jnp.exp(-0.5 * deltaXsq)
    k += (noise + jitter) * jnp.eye(x.shape[0])
    return k

In [ ]:
def M_g(M, g):
    '''
    - $M$ is a matrix with binary entries $m_{ij},$ showing whether point $j$ is in polygon $i$
    - $g$ is a vector of GP draws over grid
    - $maltmul(M, g)$ gives a vector of sums over each polygon
    '''
    M = jnp.array(M)
    g = jnp.array(g).T
    return(jnp.matmul(M, g))

In [ ]:
args = {'n_obs': jnp.array(s.n_obs),
        "x": x,
        "gp_kernel": exp_sq_kernel,
        "batch_size": 1,
        "jitter": 1e-4,
        "M_old": pol_pt_old,
        "M_new": pol_pt_new,
}

In [ ]:
def prev_model_gp_aggr(args, y=None):
    
    n_obs = args['n_obs']
    x = args["x"]
    gp_kernel=args["gp_kernel"] 
    jitter = args["jitter"]
    M_old = args["M_old"]
    M_new = args["M_new"]
    
    # random effect - aggregated GP
    length = numpyro.sample("kernel_length", dist.InverseGamma(3,3))    
    var = numpyro.sample("kernel_var", dist.HalfNormal(0.05))    
    k = gp_kernel(x, x, var, length, jitter)
    f = numpyro.sample("f", dist.MultivariateNormal(loc=jnp.zeros(x.shape[0]), covariance_matrix=k))

    # aggregate f into gp_aggr according to indexing of (point in polygon)
    gp_aggr_old = numpyro.deterministic('gp_aggr_old', M_g(M_old, f))
    gp_aggr_new = numpyro.deterministic('gp_aggr_new', M_g(M_new, f))
    gp_aggr = numpyro.deterministic('gp_aggr', jnp.concatenate([gp_aggr_old, gp_aggr_new]))
    
    # fixed effects
    b0 = numpyro.sample('b0', dist.Normal(0, 1))
    
    # linear predictor
    lp = b0 + gp_aggr
    theta = numpyro.deterministic("theta", jax.nn.sigmoid(lp))
     
    numpyro.sample("obs", dist.BinomialLogits(total_count = n_obs, logits=lp), obs=y)

In [ ]:
rng_key, rng_key_predict = random.split(random.PRNGKey(1))
prev_predictive = Predictive(prev_model_gp_aggr, num_samples=args["batch_size"])
gp_draws = prev_predictive(rng_key_predict, args)['f']

In [ ]:
rng_key, rng_key_predict = random.split(random.PRNGKey(3))
kernel = NUTS(prev_model_gp_aggr)
n_warm = 200
n_samps = 1000
mcmc_gp_aggr = MCMC(kernel, num_warmup=n_warm, num_samples=n_samps)

In [ ]:
# inference
run_mcmc = False

if run_mcmc:
    start = time.time()
    mcmc_gp_aggr.run(rng_key, args, y = jnp.array(s.y))
    t_elapsed = time.time() - start

In [ ]:
# save MCMC
#with open('MCMCs/prev_model_gp_aggr_joint_{}iter_lsIG3_3_20230214'.format(n_samps), 'wb') as file:
#    dill.dump(mcmc_gp_aggr, file)

In [ ]:
# load MCMC
#with open('MCMCs/prev_model_gp_aggr_joint_{}iter_lsIG3_3_20230214'.format(n_samps), 'rb') as file:
#    mcmc_gp_agg = dill.load(file)

In [ ]:
if run_mcmc:
    prev_samples_aggr = mcmc_gp_aggr.get_samples()
    mcmc_gp_aggr.print_summary(exclude_deterministic=False)
    
    print("\nMCMC elapsed time:", round(t_elapsed), "s")
    print("\nMCMC elapsed time:", round(t_elapsed/60), "min")
    print("\nMCMC elapsed time:", round(t_elapsed/(60*60)), "h")

    ss = numpyro.diagnostics.summary(mcmc_gp_aggr.get_samples(group_by_chain=True))
    r = np.mean(ss['gp_aggr']['n_eff'])
    print("Average ESS for all aggGP effects : " + str(round(r)))

    r = np.mean(ss['gp_aggr_old']['n_eff'])
    print("Average ESS for all aggGP-old effects : " + str(round(r)))
    print("Max r_hat for all aggGP-old effects : " + str(round(np.max(ss['gp_aggr_old']['r_hat']),2)))

    r = np.mean(ss['gp_aggr_new']['n_eff'])
    print("Average ESS for all aggGP-new effects : " + str(round(r)))
    print("Max r_hat for all aggGP-new effects : " + str(round(np.max(ss['gp_aggr_new']['r_hat']),2)))
    
    print("kernel_length R-hat : " + str(round(ss['kernel_length']['r_hat'],2)))
    print("kernel_var R-hat : " + str(round(ss['kernel_var']['r_hat'],2)))

In [ ]:
import arviz as az
prev_samples_az_gp = az.from_numpyro(mcmc_gp_aggr)
az.plot_trace(prev_samples_az_gp,  var_names=("kernel_length"))

#plt.rcParams['figure.dpi'] = 300
#plt.rcParams['savefig.dpi'] = 300
#plt.savefig('res/arviz_length.pdf')

az.plot_trace(prev_samples_az_gp,  var_names=("kernel_var"))

#plt.rcParams['figure.dpi'] = 300
#plt.rcParams['savefig.dpi'] = 300
#plt.savefig('res/arviz_var.pdf')

In [ ]:
# posterior predictive
prev_posterior_predictive_gp = Predictive(prev_model_gp_aggr, prev_samples_aggr)(random.PRNGKey(1), args)

theta_samps_gp = prev_posterior_predictive_gp['theta']

In [ ]:
theta_samps_gp.shape
theta_mean_gp = theta_samps_gp.mean(axis=0)
bci_gp_25 = np.quantile(theta_samps_gp, 0.25, axis=0)
bci_gp_75 = np.quantile(theta_samps_gp, 0.75, axis=0)

s['theta_gp'] = theta_mean_gp

theta_observed = s['estimate']
theta_estimated_gp = s['theta_gp']

_max = np.max([theta_observed, theta_estimated_gp])
_min = np.min([theta_observed, theta_estimated_gp])

n_old, _= s_old.shape
n_new, _= s_new.shape

fig, ax = plt.subplots(1, 2, figsize=(12,6))

s[0:n_old].plot(column="estimate", ax=ax[0], legend=True, cmap='plasma', vmin=_min, vmax =_max)
s[0:n_old].plot(column="theta_gp", ax=ax[1], legend=True, cmap='plasma', vmin=_min, vmax =_max)

ax[0].set_title( 'Before: Observed prevalence')
ax[1].set_title( 'Before: Estimated prevalence (aggregated GP)')

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12,6))

s[n_old:(n_old + n_new)].plot(column="estimate", ax=ax[0], legend=True, cmap='plasma', vmin=_min, vmax =_max)
s[n_old:(n_old + n_new)].plot(column="theta_gp", ax=ax[1], legend=True, cmap='plasma', vmin=_min, vmax =_max)

ax[0].set_title( 'After: Observed prevalence')
ax[1].set_title( 'After: Estimated prevalence (aggregated GP)')

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10,5))

ax[0].scatter(s[0:n_old].estimate, s[0:n_old].theta_gp)
ax[0].set_ylim(_min-0.02, _max+0.02)
ax[0].set_xlim(_min-0.02, _max+0.02)
ax[0].axline((1, 1), slope=1, ls="--", c=".3")
ax[0].set_xlabel("Observed prevalence")
ax[0].set_ylabel("Estimated prevalence")
ax[0].set_title("Before 2010")

ax[1].scatter(s[n_old:(n_old + n_new)].estimate, s[n_old:(n_old + n_new)].theta_gp)
ax[1].set_ylim(_min-0.02, _max+0.02)
ax[1].set_xlim(_min-0.02, _max+0.02)
ax[1].axline((1, 1), slope=1, ls="--", c=".3")
ax[1].set_xlabel("Observed prevalence")
ax[1].set_ylabel("Estimated prevalence, aggGP")
ax[1].set_title("After 2010")

#plt.rcParams['figure.dpi'] = 300
#plt.rcParams['savefig.dpi'] = 300
#plt.savefig('res/est_obs_gp.pdf')

# Train VAE

In [ ]:
args = {'n_obs': jnp.array(s.n_obs),
        "x": x,
        "gp_kernel": exp_sq_kernel,
        "jitter": 1e-4,
        "M_old": pol_pt_old,
        "M_new": pol_pt_new,

        # VAE training
        "rng_key": random.PRNGKey(5),
        "num_epochs": 20, 
        #"learning_rate": 1.0e-3, 
        "learning_rate": 0.0005, 
        "batch_size": 100, 
        "hidden_dim": 50, 
        "z_dim": 40, 
        "num_train": 100,
        "num_test":100,
        "vae_var": 1
}

In [ ]:
def gp_aggr(args):
    
    x = args["x"]
    gp_kernel=args["gp_kernel"] 
    jitter = args["jitter"]
    M_old = args["M_old"]
    M_new = args["M_new"]
    
    # random effect - aggregated GP
    length = numpyro.sample("kernel_length", dist.InverseGamma(3,3))        
    var = numpyro.sample("kernel_var", dist.HalfNormal(0.05))
    k = gp_kernel(x, x, var, length, jitter)
    f = numpyro.sample("f", dist.MultivariateNormal(loc=jnp.zeros(x.shape[0]), covariance_matrix=k))    
    
    # aggregate f into gp_aggr according to indexing of (point in polygon)
    gp_aggr_old = numpyro.deterministic('gp_aggr_old', M_g(M_old, f))
    gp_aggr_new = numpyro.deterministic('gp_aggr_new', M_g(M_new, f))
    gp_aggr = numpyro.deterministic('gp_aggr', jnp.concatenate([gp_aggr_old, gp_aggr_new]))

In [ ]:
# aggGP: prior predictive

rng_key, rng_key_ = random.split(random.PRNGKey(4))
agg_gp_predictive = Predictive(gp_aggr, num_samples=5)
agg_gp_preds = agg_gp_predictive(rng_key_, args)
agg_gp_draws = agg_gp_preds['gp_aggr']

In [ ]:
agg_gp_draws.shape

In [ ]:
s_old_plot = s[0:n_old].copy()

fig, axs = plt.subplots(1, 5, figsize=(20, 5))

for i in range(5):
    nm = 'gp_aggr_' + str(i)
    s_old_plot[nm] = agg_gp_draws[i,0:n_old]
    s_old_plot.plot(column=nm, ax=axs[i], legend=True)
    axs[i].set_title( 'aggGP-prior' + str(i))

In [ ]:
s_new_plot = s[n_old:(n_old + n_new)].copy()

fig, axs = plt.subplots(1, 5, figsize=(20, 5))

for i in range(5):
    nm = 'gp_aggr_' + str(i)
    s_new_plot[nm] = agg_gp_draws[i,n_old:(n_old + n_new)]
    s_new_plot.plot(column=nm, ax=axs[i], legend=True)
    axs[i].set_title( 'aggGP-prior' + str(i))

In [ ]:
def vae_encoder(hidden_dim, z_dim):
    return stax.serial(
        stax.Dense(hidden_dim, W_init=stax.randn()),
        stax.Elu,
        stax.FanOut(2),
        stax.parallel(
            stax.Dense(z_dim, W_init=stax.randn()), # mean
            stax.serial(stax.Dense(z_dim, W_init=stax.randn()), stax.Exp), # std -- i.e. diagonal covariance
        ),
    )


def vae_decoder( hidden_dim, out_dim):
    return stax.serial(
        stax.Dense(hidden_dim, W_init=stax.randn()),
        stax.Elu,
        stax.Dense(out_dim, W_init=stax.randn())
    )

def vae_model(batch, hidden_dim, z_dim):
    batch = jnp.reshape(batch, (batch.shape[0], -1))
    batch_dim, out_dim = jnp.shape(batch)
    decode = numpyro.module("decoder", vae_decoder( hidden_dim, out_dim), (batch_dim, z_dim))
    z = numpyro.sample("z", dist.Normal(jnp.zeros((z_dim,)), jnp.ones((z_dim,))))
    gen_loc = decode(z)    
    numpyro.sample("obs", dist.Normal(gen_loc, args["vae_var"]), obs=batch)
    

def vae_guide(batch, hidden_dim,  z_dim):
    batch = jnp.reshape(batch, (batch.shape[0], -1))
    batch_dim, out_dim = jnp.shape(batch)
    encode = numpyro.module("encoder", vae_encoder(hidden_dim, z_dim), (batch_dim, out_dim))
    z_loc, z_std = encode(batch)
    z = numpyro.sample("z", dist.Normal(z_loc, z_std))
    return z

In [ ]:
@jit
def epoch_train(rng_key, svi_state, num_train):

    def body_fn(i, val):
        rng_key_i = random.fold_in(rng_key, i) 
        rng_key_i, rng_key_ls, rng_key_var, rng_key_noise = random.split(rng_key_i, 4)
        loss_sum, svi_state = val # val -- svi_state
        batch = agg_gp_predictive(rng_key_i, args)['gp_aggr']        
        svi_state, loss = svi.update(svi_state, batch) 
        loss_sum += loss / args['batch_size']
        return loss_sum, svi_state

    return lax.fori_loop(0, num_train, body_fn, (0.0, svi_state)) #fori_loop(lower, upper, body_fun, init_val)

@jit
def eval_test(rng_key, svi_state, num_test):

    def body_fn(i, loss_sum):
        rng_key_i = random.fold_in(rng_key, i) 
        rng_key_i, rng_key_ls, rng_key_var, rng_key_noise = random.split(rng_key_i, 4)
        batch = agg_gp_predictive(rng_key_i, args)['gp_aggr']
        loss = svi.evaluate(svi_state, batch) / args['batch_size']
        loss_sum += loss
        return loss_sum

    loss = lax.fori_loop(0, num_test, body_fn, 0.0)
    loss = loss / num_test

    return loss

In [ ]:
adam = optim.Adam(step_size=args["learning_rate"])

svi = SVI(vae_model, vae_guide, adam, RenyiELBO(), 
          hidden_dim=args["hidden_dim"], 
          z_dim=args["z_dim"])

rng_key, rng_key_samp, rng_key_init = random.split(args["rng_key"], 3)
init_batch = agg_gp_predictive(rng_key_, args)['gp_aggr']
svi_state = svi.init(rng_key_init, init_batch)

In [ ]:
from datetime import datetime

now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

test_loss_list = []

for i in range(args['num_epochs']):
    
    rng_key, rng_key_train, rng_key_test, rng_key_infer = random.split(rng_key, 4)
    
    t_start = time.time()

    num_train = 1000
    _, svi_state = epoch_train(rng_key_train, svi_state, num_train)

    num_test = 1000
    test_loss = eval_test(rng_key_test, svi_state, num_test)
    test_loss_list += [test_loss]

    print("Epoch {}: loss = {} ({:.2f} s.)".format(i, test_loss, time.time() - t_start))
    
    if math.isnan(test_loss): break

In [ ]:
plt.figure()
plt.plot(np.arange(0, len(test_loss_list), 1), test_loss_list, label="test loss")
plt.xlabel("epochs")
plt.show()

In [ ]:
decoder_params = svi.get_params(svi_state)["decoder$params"]
args["decoder_params"] = decoder_params

In [ ]:
args["vae_var"]

In [ ]:
# Save
#with open('decoders/decoder_joint_non_std_var{}_iter20_v2'.format(args["vae_var"]), 'wb') as file:
#    pickle.dump(decoder_params, file)

In [ ]:
# Load 
#with open('decoders/decoder_joint_non_std_var1_iter20_v2', 'rb') as file:
#    decoder_params = pickle.load(file)
#    print(len(decoder_params))
    
#args["decoder_params"] = decoder_params

In [ ]:
N = init_batch.shape[1]
print(N)

In [ ]:
decoder_params = svi.get_params(svi_state)["decoder$params"]
args["decoder_params"] = decoder_params

z_dim, hidden_dim = decoder_params[0][0].shape
    
decoder_nn = vae_decoder(hidden_dim, N)  

# check output of the decoder
for i in range(5):
    z = jnp.ones(z_dim) + i
    print(decoder_nn[1](decoder_params, z)[0:4])

In [ ]:
def vae_sample(args):
    
    decoder_params = args["decoder_params"] 
    z_dim, hidden_dim = decoder_params[0][0].shape
    
    z = numpyro.sample("z", dist.Normal(jnp.zeros(z_dim), jnp.ones(z_dim)))
    decoder_nn = vae_decoder( hidden_dim, N)  
    vae_aggr = numpyro.deterministic("vae_aggr", decoder_nn[1](decoder_params, z)) 

In [ ]:
rng_key, rng_key_predict = random.split(random.PRNGKey(4))
vae_predictive = Predictive(vae_sample, num_samples=12)
vae_draws = vae_predictive(rng_key_predict, args)['vae_aggr']

In [ ]:
s_new_plot = s[n_old:(n_old + n_new)].copy()

fig, axs = plt.subplots(1, 5, figsize=(20, 5))

for i in range(5):
    nm = 'gp_aggr_' + str(i)
    s_new_plot[nm] = agg_gp_draws[i,n_old:(n_old + n_new)]
    s_new_plot.plot(column=nm, ax=axs[i], legend=True)
    axs[i].set_title( 'aggGP-prior' + str(i))

In [ ]:
# new
fig, axs = plt.subplots(2, 5, figsize=(20, 5))

for i in range(5):
    nm = 'gp_aggr_' + str(i)
    s_new_plot.plot(column=nm, ax=axs[0,i], legend=True)
    axs[0,i].set_title( 'gp_aggr-prior' + str(i))
    
for i in range(5):
    nm = 'vae_draw_' + str(i)
    s_new_plot[nm] = vae_draws[i,n_old:(n_old + n_new)]
    s_new_plot.plot(column=nm, ax=axs[1,i], legend=True)
    axs[1,i].set_title( 'VAE-prior' + str(i))

In [ ]:
(x_coords[1] - x_coords[0]) * (y_coords[40] - y_coords[39])

In [ ]:
rng_key, rng_key_ = random.split(random.PRNGKey(4))

agg_gp_predictive = Predictive(gp_aggr, num_samples=100)
agg_gp_preds = agg_gp_predictive(rng_key_, args)
agg_gp_draws = agg_gp_preds['gp_aggr']

vae_predictive = Predictive(vae_sample, num_samples=100)
vae_draws = vae_predictive(rng_key_predict, args)['vae_aggr']

m_vae = np.cov(np.transpose(vae_draws))
m_gp = np.cov(np.transpose(agg_gp_draws))

In [ ]:
cmap_choice = 'plasma'

#'plasma'
#'inferno'
fig, axs = plt.subplots(nrows=1, ncols=2, figsize = (8,4))
axs[0].imshow(m_gp, cmap=cmap_choice)
#axs[0].axis('off')
axs[0].set_title('aggrGP covariance matrix')

axs[1].imshow(m_vae, cmap=cmap_choice)
#axs[1].axis('off')
axs[1].set_title('VAE covariance matrix')

#plt.rcParams['figure.dpi'] = 300
#plt.rcParams['savefig.dpi'] = 300
#plt.savefig('res/1d_cov_matrices.pdf')

## Prevalence model - VAE

In [ ]:
def prev_model_vae_aggr(args, y=None):
    
    n_obs = args['n_obs']
    x = args["x"]
    
    # random effect
    decoder_params = args["decoder_params"] 
    z_dim, hidden_dim = decoder_params[0][0].shape
    z = numpyro.sample("z", dist.Normal(jnp.zeros(z_dim), jnp.ones(z_dim)))
    decoder_nn = vae_decoder( hidden_dim, N)  
    vae_aggr = numpyro.deterministic("vae_aggr", decoder_nn[1](decoder_params, z))
    s = numpyro.sample("sigma", dist.HalfNormal(50))
    vae = numpyro.deterministic('vae', s * vae_aggr)
    
    # fixed effects
    b0 = numpyro.sample('b0', dist.Normal(0, 1))
    lp = b0 + vae
    theta = numpyro.deterministic("theta", jax.nn.sigmoid(lp))
     
    numpyro.sample("obs", dist.BinomialLogits(total_count = n_obs, logits=lp), obs=y)

In [ ]:
if run_mcmc:

    rng_key, rng_key_predict = random.split(random.PRNGKey(6))

    # inference
    start = time.time()
    kernel = NUTS(prev_model_vae_aggr)
    mcmc_vae = MCMC(kernel, num_warmup=200, num_samples=1000)
    mcmc_vae.run(rng_key, args, y = jnp.array(s.y))
    t_elapsed = time.time() - start

    prev_samples = mcmc_vae.get_samples()
    mcmc_vae.print_summary(exclude_deterministic=False)
    
    print("\nMCMC elapsed time:", round(t_elapsed), "s")

    ss = numpyro.diagnostics.summary(mcmc_vae.get_samples(group_by_chain=True))
    r = np.mean(ss['vae_aggr']['n_eff'])
    print("Average ESS for all aggVAE effects : " + str(round(r)))
    
    ss = numpyro.diagnostics.summary(mcmc_vae.get_samples(group_by_chain=True))

    ess_old = np.mean(ss['vae_aggr']['n_eff'][0:n_old])
    r_hat_old = np.max(ss['vae_aggr']['r_hat'][0:n_old])

    ess_new = np.mean(ss['vae_aggr']['n_eff'][n_old:(n_old+n_new)])
    r_hat_new = np.max(ss['vae_aggr']['r_hat'][n_old:(n_old+n_new)])

    print("Average ESS for all aggVAE-old effects : " + str(round(ess_old)))
    print("Max r_hat for all aggVAE-old effects : " + str(round(r_hat_old,2)))

    print("Average ESS for all aggVAE-new effects : " + str(round(ess_new)))
    print("Max r_hat for all aggVAE-new effects : " + str(round(r_hat_new,2)))

    print("\nMCMC elapsed time:", round(t_elapsed), "s")
    ss = numpyro.diagnostics.summary(mcmc_vae.get_samples(group_by_chain=True))
    r = np.mean(ss['vae_aggr']['n_eff'])
    print("Average ESS for all VAE-CAR effects : " + str(round(r)))

In [ ]:
# posterior predictive - 1
prev_posterior_predictive_vae = Predictive(prev_model_vae_aggr, prev_samples)(random.PRNGKey(1), args)

theta_samps_vae_aggr = prev_posterior_predictive_vae['theta']
theta_mean_vae_aggr = theta_samps_vae_aggr.mean(axis=0)
bci_vae_aggr_25 = np.quantile(theta_samps_vae_aggr, 0.25, axis=0)
bci_vae_aggr_75 = np.quantile(theta_samps_vae_aggr, 0.75, axis=0)

s['theta_vae_aggr'] = theta_mean_vae_aggr

theta_observed = s['estimate']
theta_estimated_vae_aggr = s['theta_vae_aggr']

_max = np.max([theta_observed, theta_estimated_vae_aggr])
_min = np.min([theta_observed, theta_estimated_vae_aggr])

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12,6))

s[0:n_old].plot(column="estimate", ax=ax[0], legend=True, cmap='plasma', vmin=_min, vmax =_max)
s[0:n_old].plot(column="theta_vae_aggr", ax=ax[1], legend=True, cmap='plasma', vmin=_min, vmax =_max)

ax[0].set_title( 'Before: Observed prevalence')
ax[1].set_title( 'Before: Estimated prevalence (aggregated VAE)')

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12,6))

s[n_old:(n_old + n_new)].plot(column="estimate", ax=ax[0], legend=True, cmap='plasma', vmin=_min, vmax =_max)
s[n_old:(n_old + n_new)].plot(column="theta_vae_aggr", ax=ax[1], legend=True, cmap='plasma', vmin=_min, vmax =_max)

ax[0].set_title( 'After: Observed prevalence')
ax[1].set_title( 'After: Estimated prevalence (aggregated VAE)')

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10,5))

ax[0].scatter(s[0:n_old].estimate, s[0:n_old].theta_vae_aggr)
ax[0].set_ylim(_min-0.02, _max+0.02)
ax[0].set_xlim(_min-0.02, _max+0.02)
ax[0].axline((1, 1), slope=1, ls="--", c=".3")
ax[0].set_xlabel("Observed prevalence")
ax[0].set_ylabel("Estimated prevalence")
ax[0].set_title("Before 2010")

ax[1].scatter(s[n_old:(n_old + n_new)].estimate, s[n_old:(n_old + n_new)].theta_vae_aggr)
ax[1].set_ylim(_min-0.02, _max+0.02)
ax[1].set_xlim(_min-0.02, _max+0.02)
ax[1].axline((1, 1), slope=1, ls="--", c=".3")
ax[1].set_xlabel("Observed prevalence")
ax[1].set_ylabel("Estimated prevalence")
ax[1].set_title("After 2010")

#plt.rcParams['figure.dpi'] = 300
#plt.rcParams['savefig.dpi'] = 300
#plt.savefig('res/est_obs_vae.pdf')

In [ ]:
def prev_model_gp_aggr(args, y=None):
    
    n_obs = args['n_obs']
    x = args["x"]
    gp_kernel=args["gp_kernel"] 
    jitter = args["jitter"]
    M_old = args["M_old"]
    M_new = args["M_new"]
    
    # random effect - aggregated GP
    length = numpyro.sample("kernel_length", dist.InverseGamma(3,3))    
    var = numpyro.sample("kernel_var", dist.HalfNormal(0.05))    
    k = gp_kernel(x, x, var, length, jitter)
    f = numpyro.sample("f", dist.MultivariateNormal(loc=jnp.zeros(x.shape[0]), covariance_matrix=k))

    # aggregate f into gp_aggr according to indexing of (point in polygon)
    gp_aggr_old = numpyro.deterministic('gp_aggr_old', M_g(M_old, f))
    gp_aggr_new = numpyro.deterministic('gp_aggr_new', M_g(M_new, f))
    gp_aggr = numpyro.deterministic('gp_aggr', jnp.concatenate([gp_aggr_old, gp_aggr_new]))
    
    # fixed effects
    b0 = numpyro.sample('b0', dist.Normal(0, 1))
    
    # linear predictor
    lp = b0 + gp_aggr
    theta = numpyro.deterministic("theta", jax.nn.sigmoid(lp))
     
    numpyro.sample("obs", dist.BinomialLogits(total_count = n_obs, logits=lp), obs=y)

In [ ]:
# Load 
#with open('MCMCs/prev_model_gp_aggr_joint_500iter_lsIG3_3', 'rb') as file:
#    mcmc_gp_aggr = pickle.load(file)

#prev_samples_aggr = mcmc_gp_aggr.get_samples()
#mcmc_gp_aggr.print_summary(exclude_deterministic=False)

In [ ]:
# posterior predictive
prev_posterior_predictive = Predictive(prev_model_gp_aggr, prev_samples_aggr)(random.PRNGKey(1), args)

theta_samps_gp = prev_posterior_predictive['theta']
theta_mean_gp = theta_samps_gp.mean(axis=0)
bci_gp_25 = np.quantile(theta_samps_gp, 0.25, axis=0)
bci_gp_75 = np.quantile(theta_samps_gp, 0.75, axis=0)

s['theta_gp'] = theta_mean_gp
theta_estimated_gp = s['theta_gp']

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10,5))

ax[0].scatter(s[0:n_old].theta_gp, s[0:n_old].theta_vae_aggr)
ax[0].set_ylim(_min-0.02, _max+0.02)
ax[0].set_xlim(_min-0.02, _max+0.02)
ax[0].axline((1, 1), slope=1, ls="--", c=".3")
ax[0].set_xlabel("estimated prev (aggGP)")
ax[0].set_ylabel("estimated prev (aggVAE)")
ax[0].set_title("before 2010")

ax[1].scatter(s[n_old:(n_old + n_new)].theta_gp, s[n_old:(n_old + n_new)].theta_vae_aggr)
ax[1].set_ylim(_min-0.02, _max+0.02)
ax[1].set_xlim(_min-0.02, _max+0.02)
ax[1].axline((1, 1), slope=1, ls="--", c=".3")
ax[1].set_xlabel("estimated prev (aggGP)")
ax[1].set_ylabel("estimated prev (aggVAE)")
ax[1].set_title("after 2010")

In [ ]:
# 1
_max = np.max([s['estimate'], s['theta_gp'], s['theta_vae_aggr']])
_min = np.min([s['estimate'], s['theta_gp'], s['theta_vae_aggr']])

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(18,6))
s[0:n_old].plot(column="estimate", ax=ax[0], legend=True, cmap='plasma', vmin=_min, vmax =_max)
s[0:n_old].plot(column="theta_gp", ax=ax[1], legend=True, cmap='plasma', vmin=_min, vmax =_max)
s[0:n_old].plot(column="theta_vae_aggr", ax=ax[2], legend=True, cmap='plasma', vmin=_min, vmax =_max)
ax[0].set_title( 'Before: Observed prev')
ax[1].set_title( 'Before: Estimated prev (aggGP)')
ax[2].set_title( 'Before: Estimated prev (aggVAE)')

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(18,6))
s[n_old:(n_old+n_new)].plot(column="estimate", ax=ax[0], legend=True, cmap='plasma', vmin=_min, vmax =_max)
s[n_old:(n_old+n_new)].plot(column="theta_gp", ax=ax[1], legend=True, cmap='plasma', vmin=_min, vmax =_max)
s[n_old:(n_old+n_new)].plot(column="theta_vae_aggr", ax=ax[2], legend=True, cmap='plasma', vmin=_min, vmax =_max)
ax[0].set_title( 'After: Observed prev')
ax[1].set_title( 'After: Estimated prev (aggGP)')
ax[2].set_title( 'After: Estimated prev (aggVAE)')